# R-CNN SVM Training

Train linear SVMs with hard negative mining for aircraft detection.

## 1. Setup and Imports

In [2]:
from __future__ import annotations

import sys
import pickle
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.decomposition import PCA
import torch

sys.path.insert(0, str(Path.cwd() / "rcnn"))

from rcnn.config.config import RCNNConfig
from rcnn.train_svm import SVMTrainer
from rcnn.data import AirbusRCNNDataset

sns.set_context("talk")
sns.set_style("whitegrid")

if torch.cuda.is_available():
    DEVICE = "cuda"
    print(f"Using device: {DEVICE}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
elif torch.backends.mps.is_available():
    DEVICE = "mps"
    print(f"Using device: {DEVICE}")
    print("Apple Silicon GPU acceleration enabled")
else:
    DEVICE = "cpu"
    print(f"Using device: {DEVICE}")
    print("Note: Training will be slow on CPU")

print("\n✓ Setup complete")

Using device: mps
Apple Silicon GPU acceleration enabled

✓ Setup complete


## 2. Configuration

In [3]:
# Paths
BASE_DIR = Path("dataset/airbus-aircrafts-sample-dataset")
ANNOTATIONS_CSV = BASE_DIR / "annotations.csv"
IMAGES_DIR = BASE_DIR / "images"
ARTIFACTS_DIR = Path("artifacts/rcnn")

assert ANNOTATIONS_CSV.exists(), f"Annotations not found: {ANNOTATIONS_CSV}"
assert IMAGES_DIR.exists(), f"Images directory not found: {IMAGES_DIR}"

config = RCNNConfig()

print("SVM Training Configuration:")
print(f"  Fine-tuned model: {ARTIFACTS_DIR / 'finetuned_cnn_best.pth'}")
print(f"  Feature layer: {config.feature_layer} (4096-dim)")
print(f"\nSVM criteria (stricter than fine-tuning):")
print(f"  Positive IoU: {config.svm_pos_iou_threshold} (GT boxes only)")
print(f"  Negative IoU: < {config.svm_neg_iou_threshold}")
print(f"  Hard negative mining: {config.svm_hard_neg_mining}")
print(f"\nDataset:")
print(f"  Annotations: {ANNOTATIONS_CSV}")
print(f"  Images: {IMAGES_DIR}")

print("\n✓ Configuration loaded")

SVM Training Configuration:
  Fine-tuned model: artifacts/rcnn/finetuned_cnn_best.pth
  Feature layer: fc7 (4096-dim)

SVM criteria (stricter than fine-tuning):
  Positive IoU: 1.0 (GT boxes only)
  Negative IoU: < 0.3
  Hard negative mining: True

Dataset:
  Annotations: dataset/airbus-aircrafts-sample-dataset/annotations.csv
  Images: dataset/airbus-aircrafts-sample-dataset/images

✓ Configuration loaded


## 3. Initialize SVM Trainer

**This cell defines:** `trainer` (needed by later cells)

In [3]:
trainer = SVMTrainer(config, device=DEVICE)

print("\n✓ SVM Trainer initialized")
print(f"  Fine-tuned CNN loaded: {(ARTIFACTS_DIR / 'finetuned_cnn_best.pth').exists()}")

Loading fine-tuned CNN...
✓ Loaded fine-tuned model from artifacts/rcnn/finetuned_cnn_best.pth

✓ SVM Trainer initialized
  Fine-tuned CNN loaded: True


## 4. Train SVM

**This cell defines:** `svm` (the trained SVM classifier)

In [6]:
annotations_full = pd.read_csv(ANNOTATIONS_CSV)
annotations_subset = annotations_full[annotations_full['image_id'].isin(
    annotations_full['image_id'].unique()[:20]
)]
subset_csv = BASE_DIR.parent / "temp_annotations.csv"
annotations_subset.to_csv(subset_csv, index=False)

In [4]:
trainer.train(
    annotations_csv=subset_csv,
    images_dir=IMAGES_DIR,
    batch_size=128,
    num_workers=0
)

R-CNN Stage 3: SVM Training

Creating SVM training dataset...
Note: Using stricter IoU thresholds than fine-tuning
  Positives: IoU ≈ 1.0 (ground-truth boxes only)
  Negatives: IoU < 0.3
Building proposal index for stage: svm
Total images to process: 20

[1/20] Processing: 4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg (generating proposals...) done
[2/20] Processing: 74d335db-bf64-424a-88d7-5c24625c50b1.jpg (generating proposals...) done
[3/20] Processing: 4e8c95f3-bbb6-4e2f-920d-8d3b2bd1d29f.jpg (generating proposals...) done
[4/20] Processing: 72fa2a77-cf8e-44a2-9ab2-f11ca63b4b72.jpg (generating proposals...) done
[5/20] Processing: 2314c1b5-ec8f-4212-b42f-43365a13fd20.jpg (generating proposals...) done
[6/20] Processing: 0641acc3-c0b9-4f9d-b0ba-7ad18aa08864.jpg (generating proposals...) done
[7/20] Processing: bed88cf9-7887-4bfd-a1d7-b5499ee24f03.jpg (generating proposals...) done
[8/20] Processing: 3463b88d-1e11-4c04-9868-e8c72d510556.jpg (generating proposals...) done
[9/20] Processing

/Users/danylolitvinchuk/python-workspace/NN-Lab2/myenv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Extracting features:   0%|          | 0/153 [00:00<?, ?it/s]

✓ Extracted features: (19514, 4096)

SVM training data for class 1:
  Positives (IoU ≈ 1.0): 1
  Negatives (IoU < 0.3): 19513
  Ignored (0.3 ≤ IoU < 1.0): 0

Training SVM for class 1...
  Using subset of negatives: 10/19513
  Initial training: 1 pos + 10 neg
  Hard negative mining...
  Found 41 hard negatives
  Retraining with 52 samples...
  ✓ Training accuracy: 1.0000

✓ Saved SVM to artifacts/rcnn/svm_class_1.pkl
✓ Saved all SVMs to artifacts/rcnn/svms_all.pkl

SVM Training Complete!


In [5]:
# trainer.train(
#     annotations_csv=ANNOTATIONS_CSV,
#     images_dir=IMAGES_DIR,
#     batch_size=64,
#     num_workers=4
# )

svm = trainer.svms[1]  # Aircraft SVM (class_id=1)

print(f"\n{'='*60}")
print("✓ SVM training complete!")
print(f"✓ SVM ready for evaluation")
print(f"{'='*60}")


✓ SVM training complete!
✓ SVM ready for evaluation


## 5. Load Trained SVM (Optional)

In [4]:
svm_path = ARTIFACTS_DIR / "svm_class_1.pkl"
if svm_path.exists():
    with open(svm_path, 'rb') as f:
        svm = pickle.load(f)
    print(f"✓ Loaded SVM from {svm_path}")
else:
    print(f"⚠️  SVM not found at {svm_path}")

print("Skipping reload (using freshly trained SVM from above)")

✓ Loaded SVM from artifacts/rcnn/svm_class_1.pkl
Skipping reload (using freshly trained SVM from above)


## 6. Extract Features for Visualization

In [7]:
print("Extracting features for visualization and comparison...")
print("(This uses a smaller dataset for speed)\n")

vis_dataset = AirbusRCNNDataset(
    annotations_csv=subset_csv,
    images_dir=IMAGES_DIR,
    stage="svm",
    num_proposals=500,
    cache_proposals=True,
)

vis_features, vis_labels, vis_ious = trainer.extract_features(
    vis_dataset,
    batch_size=64,
    num_workers=4
)

pos_mask = (vis_labels == 1) & (vis_ious >= 0.99)
neg_mask = vis_ious < 0.3

print("\nFeature extraction complete:")
print(f"  Total samples: {len(vis_features)}")
print(f"  Positives (IoU ≈ 1.0): {pos_mask.sum()}")
print(f"  Negatives (IoU < 0.3): {neg_mask.sum()}")
print(f"  Ignored (0.3 ≤ IoU < 1.0): {len(vis_features) - pos_mask.sum() - neg_mask.sum()}")

print("\n✓ Features ready for visualization and comparison")

Extracting features for visualization and comparison...
(This uses a smaller dataset for speed)

Building proposal index for stage: svm
Total images to process: 20

[1/20] Processing: 4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg (generating proposals...) done
[2/20] Processing: 74d335db-bf64-424a-88d7-5c24625c50b1.jpg (generating proposals...) done
[3/20] Processing: 4e8c95f3-bbb6-4e2f-920d-8d3b2bd1d29f.jpg (generating proposals...) done
[4/20] Processing: 72fa2a77-cf8e-44a2-9ab2-f11ca63b4b72.jpg (generating proposals...) done
[5/20] Processing: 2314c1b5-ec8f-4212-b42f-43365a13fd20.jpg (generating proposals...) done
[6/20] Processing: 0641acc3-c0b9-4f9d-b0ba-7ad18aa08864.jpg (generating proposals...) done
[7/20] Processing: bed88cf9-7887-4bfd-a1d7-b5499ee24f03.jpg (generating proposals...) done
[8/20] Processing: 3463b88d-1e11-4c04-9868-e8c72d510556.jpg (generating proposals...) done
[9/20] Processing: 1e7e0450-6eb3-479e-88c2-990abc8207fa.jpg (generating proposals...) done
[10/20] Processi

NameError: name 'trainer' is not defined

## 7. Visualize Feature Space (PCA Projection)

In [ ]:
sample_features = np.vstack([
    vis_features[pos_mask][:200],
    vis_features[neg_mask][:200]
])
sample_labels = np.concatenate([
    np.ones(min(200, pos_mask.sum())),
    np.zeros(min(200, neg_mask.sum()))
])

pca = PCA(n_components=2)
features_2d = pca.fit_transform(sample_features)

fig, ax = plt.subplots(figsize=(10, 8))

colors = ['red', 'green']
labels_text = ['Background', 'Aircraft']

for label in [0, 1]:
    mask = sample_labels == label
    ax.scatter(
        features_2d[mask, 0],
        features_2d[mask, 1],
        c=colors[label],
        label=labels_text[label],
        alpha=0.6,
        s=50
    )

ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
ax.set_title('SVM Training Samples (PCA 2D Projection)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(ARTIFACTS_DIR / 'svm_feature_space.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✓ Saved to: {ARTIFACTS_DIR / 'svm_feature_space.png'}")

## 8. Compare SVM vs CNN Softmax Classifier

In [ ]:
from rcnn.models.cnn import AlexNetFeatureExtractor

print("Loading fine-tuned CNN for comparison...")

cnn_classifier = AlexNetFeatureExtractor(
    pretrained=True,
    feature_layer=config.feature_layer,
    num_classes=config.num_classes
)

checkpoint = torch.load(
    ARTIFACTS_DIR / 'finetuned_cnn_best.pth',
    map_location=DEVICE,
    weights_only=False
)
cnn_classifier.load_state_dict(checkpoint['model_state_dict'])
cnn_classifier = cnn_classifier.to(DEVICE)
cnn_classifier.eval()

print("✓ CNN classifier loaded")
print("\nComputing predictions...")

with torch.no_grad():
    svm_predictions = svm.predict(vis_features)
    svm_scores = svm.decision_function(vis_features)
    
    vis_features_torch = torch.from_numpy(vis_features).float().to(DEVICE)
    cnn_logits = cnn_classifier.classifier(vis_features_torch)
    cnn_predictions = cnn_logits.argmax(dim=1).cpu().numpy()

svm_mask = pos_mask | neg_mask
svm_labels = vis_labels[svm_mask]
svm_preds = svm_predictions[svm_mask]
cnn_preds = cnn_predictions[svm_mask]

print("\n" + "="*60)
print("CLASSIFICATION COMPARISON (on SVM validation set)")
print("="*60)

print("\n📊 SVM Classifier:")
print(classification_report(
    svm_labels, svm_preds,
    target_names=['Background', 'Aircraft'],
    digits=4
))

print("\n📊 CNN Softmax Classifier:")
print(classification_report(
    svm_labels, cnn_preds,
    target_names=['Background', 'Aircraft'],
    digits=4
))

svm_acc = (svm_preds == svm_labels).mean()
cnn_acc = (cnn_preds == svm_labels).mean()

print("\n" + "="*60)
print(f"SVM Accuracy:     {svm_acc:.4f}")
print(f"CNN Accuracy:     {cnn_acc:.4f}")
print(f"Improvement:      {(svm_acc - cnn_acc)*100:+.2f}%")
print("="*60)

## 9. Confusion Matrices Comparison

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

svm_cm = confusion_matrix(svm_labels, svm_preds)
sns.heatmap(
    svm_cm, annot=True, fmt='d', cmap='Blues',
    xticklabels=['Background', 'Aircraft'],
    yticklabels=['Background', 'Aircraft'],
    ax=axes[0],
    cbar_kws={'label': 'Count'}
)
axes[0].set_title('SVM Confusion Matrix')
axes[0].set_ylabel('True Label')
axes[0].set_xlabel('Predicted Label')

# CNN confusion matrix
cnn_cm = confusion_matrix(svm_labels, cnn_preds)
sns.heatmap(
    cnn_cm, annot=True, fmt='d', cmap='Greens',
    xticklabels=['Background', 'Aircraft'],
    yticklabels=['Background', 'Aircraft'],
    ax=axes[1],
    cbar_kws={'label': 'Count'}
)
axes[1].set_title('CNN Softmax Confusion Matrix')
axes[1].set_ylabel('True Label')
axes[1].set_xlabel('Predicted Label')

plt.tight_layout()
plt.savefig(ARTIFACTS_DIR / 'svm_vs_cnn_confusion.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✓ Saved to: {ARTIFACTS_DIR / 'svm_vs_cnn_confusion.png'}")

## 10. Summary

In [ ]:
print("="*60)
print("SVM TRAINING SUMMARY")
print("="*60)

annotations = pd.read_csv(ANNOTATIONS_CSV)

print(f"\n📁 Dataset:")
print(f"  Total images: {annotations['image_id'].nunique()}")
print(f"  Total annotations: {len(annotations)}")

print(f"\n🔧 SVM Configuration:")
print(f"  Feature extractor: Fine-tuned AlexNet (fc7, 4096-dim)")
print(f"  Positive samples: IoU = 1.0 (GT boxes only)")
print(f"  Negative samples: IoU < {config.svm_neg_iou_threshold}")
print(f"  Hard negative mining: {config.svm_hard_neg_mining}")

print(f"\n📊 Results:")
print(f"  SVM accuracy:     {svm_acc:.4f} ({svm_acc*100:.2f}%)")
print(f"  CNN accuracy:     {cnn_acc:.4f} ({cnn_acc*100:.2f}%)")
print(f"  Improvement:      {(svm_acc - cnn_acc)*100:+.2f}%")

print(f"\n💾 Saved Artifacts:")
artifacts = [
    'svm_class_1.pkl',
    'svms_all.pkl',
    'svm_feature_space.png',
    'svm_vs_cnn_confusion.png'
]
for artifact in artifacts:
    path = ARTIFACTS_DIR / artifact
    if path.exists():
        size_mb = path.stat().st_size / (1024 * 1024)
        print(f"  ✓ {artifact} ({size_mb:.1f} MB)")
    else:
        print(f"  ✗ {artifact} (not found)")

print("\n" + "="*60)
print("✅ STAGE 3 COMPLETE!")
print("="*60)
print("\n📋 Next Steps:")
print("  1. Train bounding box regressor (Stage 4)")
print("  2. Implement inference pipeline")
print("  3. Evaluate mAP on test set")
print("="*60)